<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [12]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [15]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [16]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [17]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [18]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

Running on local machine

In [19]:
# Initialize MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("Anomaly Detection")


for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/08/21 00:45:07 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 00:45:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/572329765040959268/runs/12fcb366a4e241c4afcf77aabf179a9f.
2024/08/21 00:45:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/572329765040959268.
2024/08/21 00:45:08 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 00:45:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/572329765040959268/runs/306ba86c05ed4d39a439ec653c3f4251.
2024/08/21 00:45:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5723

### Register the Model

In [35]:
# dagshub setup

import dagshub
dagshub.init(repo_owner='skpatil.ai', repo_name='MLFlow-Dagshub-Tutorial', mlflow=True)

Initialized MLflow to track repo "skpatil.ai/MLFlow-Dagshub-Tutorial"

Repository skpatil.ai/MLFlow-Dagshub-Tutorial initialized!

In [42]:
# Ideally you will not require following 4 lines if you have started fresh and do not have any previous dagshub credentials on your computer
import os

os.environ['MLFLOW_TRACKING_USERNAME'] = 'skpatil.ai' 
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow' 
os.environ['MLFLOW_TRACKING_PASSWORD'] = '3ad69129c3f88b0e75f3586c948787e32c1ed105'



# Initialize MLflow
mlflow.set_tracking_uri("https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow")
mlflow.set_experiment("Anomaly Detection")


for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/08/21 01:44:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 01:44:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow/#/experiments/0/runs/bc80f117644640fbb88e8e513de7818a.
2024/08/21 01:44:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow/#/experiments/0.
2024/08/21 01:45:06 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 01:45:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow/#/experiments/0/runs/7ca35dd356c0457d957f0c5b835bf3f3.
2024/08/21 01:45:07 INFO mlflow.tracking._tracking_service.c

# Run using Dagshub

In [43]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'XGB-Smote'.
2024/08/21 01:46:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.
2024/08/21 01:46:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow/#/experiments/0/runs/a428e1b0a70a4027931dbb15fa60542b.
2024/08/21 01:46:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/skpatil.ai/MLFlow-Dagshub-Tutorial.mlflow/#/experiments/0.


### Load the Model

### Transition the Model to Production

In [50]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1724230694458, current_stage='None', description='', last_updated_timestamp=1724230694458, name='anomaly-detection-prod', run_id='a428e1b0a70a4027931dbb15fa60542b', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 